In [16]:
import torch
# from transformers import BertModel, BertTokenizer
# from kobert_transformers import get_tokenizer
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

from collections import Counter
import tqdm as notebook_tqdm

import pandas as pd #결과값을 데이터프레임 객체로 저장하기 위해 이용


In [17]:
# # Load the KoBERT model and tokenizer
# model_name = 'monologg/kobert'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# Load the KoGPT model and tokenizer
model_name = 'skt/kogpt2-base-v2'
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

/home/arkwith/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [18]:
# Function to tokenize text using KoBERT
def tokenize(text):
    tokens = tokenizer.tokenize(text)
    return tokens

In [19]:
# data 불러오기
data = pd.read_csv('youtube_channel_comments_cleaneddata.csv')
data.head()


,channelId,channelTitle,channelPublishedAt,subscriberCount,videoId,videoTitle,videoAuthorId,videoPublishedAt,duration,viewCount,likeCount,dislikeCount,commentId,commentAuthor,authorId,commentText,commentLikeCount,commentPublishedAt,parentCommentId,cleaned_commentText
0,UCMFk5S7g5DY-CZNVh_Kyz_A,약사가 들려주는 약 이야기,2017-03-02 02:28:38+00:00,1780000,qlLkiUU0o1A,우리 몸에 대장암이 생길때 나타나는 위험신호 대장암위험신호,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,56,19765,339,0,UgxIwrowVdyFJFZloPx4AaABAg,@user-en3wv3ng8q,UCJZMfxEsrCsfkECnVQx1J5g,야채 많이드시고 가끔 요구르트 챙겨드시면\n어느정도 예방됨,0,2024-06-05 19:32:41+00:00,NaN,야채 많이 드시고 가끔 요구르트 챙겨드시면 어느 정도 예방됨
1,UCMFk5S7g5DY-CZNVh_Kyz_A,약사가 들려주는 약 이야기,2017-03-02 02:28:38+00:00,1780000,qlLkiUU0o1A,우리 몸에 대장암이 생길때 나타나는 위험신호 대장암위험신호,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,56,19765,339,0,UgzzY7tk7b81qVjPlu14AaABAg,@user-fb4hd5lg1v,UCtTgUzxcR7KXdKUG4k3qTLA,제일 확실한건 변비 설사 반복 됩니다 꼭 검사해보셔요,2,2024-06-05 11:11:22+00:00,NaN,제일 확실한 건 변비 설사 반복 됩니다 꼭 검사해 보셔요
2,UCMFk5S7g5DY-CZNVh_Kyz_A,약사가 들려주는 약 이야기,2017-03-02 02:28:38+00:00,1780000,qlLkiUU0o1A,우리 몸에 대장암이 생길때 나타나는 위험신호 대장암위험신호,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,56,19765,339,0,UgwWLF6ds6wSe2lU6DN4AaABAg,@inuyasha1017,UCEAlDUqmaikJpmie3nAIozg,좋은정보 감사합니다 😊😊😊,0,2024-06-05 10:43:46+00:00,NaN,좋은 정보 감사합니다
3,UCMFk5S7g5DY-CZNVh_Kyz_A,약사가 들려주는 약 이야기,2017-03-02 02:28:38+00:00,1780000,qlLkiUU0o1A,우리 몸에 대장암이 생길때 나타나는 위험신호 대장암위험신호,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,56,19765,339,0,UgxDtXAN64oybfbaEDF4AaABAg,@user-cj6rk5qn9o,UC2HHPaUByk-R9ykbP3yjsaQ,걸리면 하느님 만나로 올라갈게요,1,2024-06-05 10:11:54+00:00,NaN,걸리면 하느님 만나로 올라갈게요
4,UCMFk5S7g5DY-CZNVh_Kyz_A,약사가 들려주는 약 이야기,2017-03-02 02:28:38+00:00,1780000,qlLkiUU0o1A,우리 몸에 대장암이 생길때 나타나는 위험신호 대장암위험신호,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,56,19765,339,0,Ugx946j7jz6kQr-vDsh4AaABAg,@user-ye5ne4mc6d,UCQ0eOpBzyy1vgyF4nUTAb_Q,이중에 하나도 해당 안됐는데\n대장암 4기..,10,2024-06-05 08:33:27+00:00,NaN,이 중에 하나도 해당 안 됐는데 대장암 4기


In [20]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56405 entries, 0 to 56404
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   channelId            56405 non-null  object
 1   channelTitle         56405 non-null  object
 2   channelPublishedAt   56405 non-null  object
 3   subscriberCount      56405 non-null  int64 
 4   videoId              56405 non-null  object
 5   videoTitle           56405 non-null  object
 6   videoAuthorId        56405 non-null  object
 7   videoPublishedAt     56405 non-null  object
 8   duration             56405 non-null  int64 
 9   viewCount            56405 non-null  int64 
 10  likeCount            56405 non-null  int64 
 11  dislikeCount         56405 non-null  int64 
 12  commentId            56405 non-null  object
 13  commentAuthor        56277 non-null  object
 14  authorId             56405 non-null  object
 15  commentText          56405 non-null  object
 16  comm

In [21]:
# data를 채널아이디와 정제된 댓글만 남김
data = data[['channelId', 'videoPublishedAt','cleaned_commentText']]
data.head()
# CSV 파일로 저장
# data.to_csv('cleaned_comments.csv', index=False, encoding='utf-8-sig')

,channelId,videoPublishedAt,cleaned_commentText
0,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,야채 많이 드시고 가끔 요구르트 챙겨드시면 어느 정도 예방됨
1,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,제일 확실한 건 변비 설사 반복 됩니다 꼭 검사해 보셔요
2,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,좋은 정보 감사합니다
3,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,걸리면 하느님 만나로 올라갈게요
4,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,이 중에 하나도 해당 안 됐는데 대장암 4기


In [22]:
# data 크기확인
data.shape

(56405, 3)

In [23]:
# soynlp를 적용하기 위해 데이터 훈련
# NaN 값 제거
data = data.dropna(subset=['cleaned_commentText'])

# 또는

# NaN 값 공백 문자열로 대체
data['cleaned_commentText'] = data['cleaned_commentText'].fillna('')

# Extract comments for each channel
comments_by_channel = {}
for channel in data['channelId'].unique():
    comments = data[data['channelId'] == channel]['cleaned_commentText']
    comments_text = ' '.join(comments)
    comments_by_channel[channel] = comments_text



In [24]:
# Tokenize and count words for each channel
keywords_by_channel_kobert = {}
for channel, text in comments_by_channel.items():
    tokens = tokenize(text)
    counter = Counter(tokens)
    keywords_by_channel_kobert[channel] = counter.most_common(10)  # Top 10 keywords

In [25]:
# Display the keywords for each channel
keywords_by_channel_kobert

{'UCMFk5S7g5DY-CZNVh_Kyz_A': [('다', 32447),
  ('요', 26111),
  ('니', 21248),
  ('▁', 11725),
  ('습니', 10416),
  ('합', 9897),
  ('▁감사', 8537),
  ('서', 7277),
  ('도', 7161),
  ('셔', 6391)],
 'UCY-mXLM6DsS9cmSwlh0tqSA': [('요', 6062),
  ('▁', 5549),
  ('다', 5432),
  ('니', 3445),
  ('ht', 3038),
  ('t', 3015),
  ('ps', 2835),
  ('y', 2587),
  ('out', 2285),
  ('u', 2230)],
 'UC3iSLVH0MxHfwO69oHKpvog': [('요', 6716),
  ('다', 5818),
  ('니', 3497),
  ('▁', 3382),
  ('도', 2065),
  ('습니', 1936),
  ('합', 1509),
  ('▁안', 1473),
  ('▁먹', 1366),
  ('▁감사', 1312)],
 'UC6ggXTuBVchhwHeQ12Ita1w': [('요', 2710),
  ('다', 2475),
  ('니', 1633),
  ('▁', 923),
  ('습니', 805),
  ('세', 720),
  ('합', 713),
  ('▁감사', 699),
  ('님', 652),
  ('▁복용', 644)],
 'UCCMFTDGarjgZLc1DlIbbvRg': [('요', 362),
  ('다', 195),
  ('▁', 195),
  ('ᄒ', 190),
  ('니', 155),
  ('도', 92),
  ('▁감사', 88),
  ('합', 83),
  ('님', 75),
  ('네', 68)]}

In [19]:
# 텍스트 토크나이저 적용

# data['tokenized'] = data['cleaned_commentText'].apply(okt_tokenize)
# soynlp의 LTokenizer를 이용한 토크나이저 적용
data['tokenized'] = data['cleaned_commentText'].apply(tokenize_with_stopwords)
data.head()



,channelId,videoPublishedAt,cleaned_commentText,tokenized
0,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,야채 많이 드시고 가끔 요구르트 챙겨드시면 어느 정도 예방됨,"[야채, 가끔, 요구르트, 챙겨, 드시면, 어느, 예방]"
1,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,제일 확실한 건 변비 설사 반복 됩니다 꼭 검사해 보셔요,"[제일, 확실, 변비, 설사, 반복, 검사]"
2,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,좋은 정보 감사합니다,"[정보, 감사]"
3,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,걸리면 하느님 만나로 올라갈게요,"[걸리, 하느님, 만나로, 올라, 갈게요]"
4,UCMFk5S7g5DY-CZNVh_Kyz_A,2024-06-05 08:18:39+00:00,이 중에 하나도 해당 안 됐는데 대장암 4기,"[해당, 대장암, 4기]"


In [20]:
# 채널별로 데이터 그룹화 및 토큰 리스트 합치기
grouped_data = data.groupby('channelId')['tokenized'].agg(lambda x: [token for sublist in x for token in sublist])

# 각 채널의 토큰 리스트를 사용하여 문서-단어 행렬을 생
# CountVectorizer 초기화 (토큰 리스트를 직접 처리하도록 설정)
vectorizer = CountVectorizer(tokenizer=lambda x: x, lowercase=False, preprocessor=lambda x: x)


In [21]:
grouped_data

channelId
UC3iSLVH0MxHfwO69oHKpvog    [리틀, 약사님, 최고, 이십니다, 감사, 비문증, 브로, 멜라, 파파인에, 대해서...
UC6ggXTuBVchhwHeQ12Ita1w    [Ph, ytosome, 베르, 베린, 복용, 방법, 안내, 복용, 이미, 아시겠지...
UCCMFTDGarjgZLc1DlIbbvRg    [갑니다, 감사, se, up, the, su, btitles, to, autotr...
UCMFk5S7g5DY-CZNVh_Kyz_A    [야채, 가끔, 요구르트, 챙겨, 드시면, 어느, 예방, 제일, 확실, 변비, 설사...
UCY-mXLM6DsS9cmSwlh0tqSA    [담백, 하루, 아르기닌, 변비, 유산균, 공구, 오픈, 구매, 링크, 트랜짓, 유...
Name: tokenized, dtype: object

In [22]:
from collections import Counter

# 각 채널별로 상위 20개 단어 추출
top_words_per_channel = {}
for channel_id, tokens in grouped_data.items():
    # 토큰의 빈도수 계산
    token_counts = Counter(tokens)
    # 가장 빈번한 30개 단어 추출
    top_words = token_counts.most_common(30)
    top_words_per_channel[channel_id] = top_words

In [23]:
# 결과 출력
for channel_id, words in top_words_per_channel.items():
    print(f"Channel {channel_id}:")
    for word, count in words:
        print(f"{word}: {count}")
    print("\n")

Channel UC3iSLVH0MxHfwO69oHKpvog:
감사: 1317
비타: 626
마그네슘: 613
제품: 599
영양제: 573
리틀: 552
복용: 538
영상: 521
밴드: 516
약사님: 491
오메가: 484
약사: 451
섭취: 403
건강: 401
효과: 358
도움: 348
하루: 336
정보: 333
답변: 309
유튜: 302
생각: 291
자세: 291
음식: 283
사람: 274
문제: 270
이용: 249
불편: 246
다른: 241
커피: 239
대한: 235


Channel UC6ggXTuBVchhwHeQ12Ita1w:
감사: 706
복용: 678
약사님: 557
영상: 329
제품: 273
섭취: 268
영양제: 244
효과: 236
정보: 233
건강: 178
선생님: 176
베르: 175
멜라토: 171
피세틴: 166
도움: 164
추천: 143
미국: 133
베린: 124
비타: 123
구매: 119
하루: 118
아침: 116
레스베라트롤: 108
피부: 107
궁금: 106
라이: 104
구입: 94
함께: 87
Pharm: 83
acistPal: 83


Channel UCCMFTDGarjgZLc1DlIbbvRg:
감사: 88
약사님: 42
크림: 37
효과: 37
이지약사님: 31
제품: 30
멜라토: 25
피부: 22
정보: 21
멜라: 21
영상: 20
발라: 19
이지: 15
바르: 15
부위: 15
약국: 14
처방: 14
색소: 14
궁금: 14
쓰고: 14
오랜만: 13
발라도: 13
흡수: 12
여드름: 12
자외선: 12
흉터: 12
톡톡: 11
노사: 11
함량: 11
해주: 11


Channel UCMFk5S7g5DY-CZNVh_Kyz_A:
감사: 8550
섭취: 5693
영양제: 3680
오메가: 3674
비타: 3640
제품: 3448
약사님: 3259
복용: 3193
영상: 2882
도움: 2480
시청: 2374
추천: 2267
생각: 1848
마그네슘: 1831
말씀: 1661

In [24]:
vectorizer

CountVectorizer(lowercase=False,
                preprocessor=<function <lambda> at 0x7f5e41347b50>,
                tokenizer=<function <lambda> at 0x7f5ef754e680>)

In [25]:
# channelId별로 LDA를 위해 5개의 문서-단어 행렬 생성

document_term_matrix = vectorizer.fit_transform(grouped_data)



/home/arkwith/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [26]:
# LDA 모델 초기화 및 학습
lda_model = LatentDirichletAllocation(n_components=5, random_state=0)  # n_components는 토픽 수
lda_model.fit(document_term_matrix)

LatentDirichletAllocation(n_components=5, random_state=0)

In [27]:
# 학습된 LDA 모델로부터 각 토픽의 주요 단어들을 확인하고, 각 채널의 주요 토픽을 출력합니다.
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

tf_feature_names = vectorizer.get_feature_names_out()
print_top_words(lda_model, tf_feature_names, 10)

# 각 채널의 주요 토픽 출력
channel_topics = lda_model.transform(document_term_matrix)
for channel_idx, topic_dist in enumerate(channel_topics):
    top_topic = topic_dist.argmax()
    print(f"Channel {grouped_data.index[channel_idx]} 주요 토픽: {top_topic}")

Topic #0: 졌지만 활에 셨군요 빨간색 원으로 고름 풍요로운 안쪄요 이쁘게 당ㅋㅋㅋ
Topic #1: 감사 복용 약사님 영상 효과 제품 정보 멜라토 영양제 섭취
Topic #2: 졌지만 활에 셨군요 빨간색 원으로 고름 풍요로운 안쪄요 이쁘게 당ㅋㅋㅋ
Topic #3: 감사 1등은 영양제 추천 제품 비타 양과자 도움 구매 영상
Topic #4: 감사 섭취 비타 영양제 오메가 제품 약사님 복용 영상 도움
Channel UC3iSLVH0MxHfwO69oHKpvog 주요 토픽: 4
Channel UC6ggXTuBVchhwHeQ12Ita1w 주요 토픽: 1
Channel UCCMFTDGarjgZLc1DlIbbvRg 주요 토픽: 1
Channel UCMFk5S7g5DY-CZNVh_Kyz_A 주요 토픽: 4
Channel UCY-mXLM6DsS9cmSwlh0tqSA 주요 토픽: 3
